In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
import cartopy
import cartopy.crs as ccrs
import cmocean
import xarray as xr

# Maximum Entropy Species Distribution Modeling
- Using MaxEnt model with B-SOSE Southern Ocean State Estimate and SOPHY phytoplankton presence data to build seasonal species distribution models

In [ ]:
def so_ax():
  map_proj = ccrs.SouthPolarStereo()
  fig = plt.figure(figsize=[12, 12])  # inches
  ax = plt.subplot(projection=map_proj)
  ax.set_extent([-180, 180, -90, -40], ccrs.PlateCarree())
  ax.gridlines(linestyle='--', draw_labels=True)
  ax.add_feature(cartopy.feature.LAND)
  return ax 

## Raw B-SOSE data processing
- In: monthly averages for 5 year span (2013-2017) per environmental variable
- Out: 4 seasonal averages (fall, winter, spring, summer) per environmental variable

In [ ]:
# austral spring (September–November), summer (December–February), autumn (March–May), and winter (June–August)
# summer = [59-61], autumn = [62-64], winter = [65-67], spring = [68-70]

# mapping season to indexes in BSOSE i133_2013to2018_monthly representative of each season
seasons: dict = {'summer': [59, 60, 61], 'fall': [62, 63, 64], 'winter': [65, 66, 67], 'spring': [68, 69, 70]}

var_info = {'THETA': {'cmap': cmocean.cm.thermal, 'simple_name': 'temperature', 'file_prefix': 'Theta', 'vmin': -5},
            'SIarea': {'cmap': cmocean.cm.ice, 'simple_name': 'sea_ice_extent', 'file_prefix': 'SeaIceArea', 'vmin': 0},
            'SALT': {'cmap': cmocean.cm.haline, 'simple_name': 'salinity', 'file_prefix': 'Salt', 'vmin': 30},
            'TRAC03': {'cmap': cmocean.cm.haline, 'simple_name': 'oxygen', 'file_prefix': 'O2', 'vmin': 0},
            'TRAC04': {'cmap': cmocean.cm.matter, 'simple_name': 'nitrate', 'file_prefix': 'NO3', 'vmin': 0},
            'BLGCHL': {'cmap': cmocean.cm.algae, 'simple_name': 'chlorophyll', 'file_prefix': 'Chl', 'vmin': 0}}

In [ ]:
fig, axes = plt.subplots(len(var_info), len(seasons), figsize=(12, 12), subplot_kw={'projection': ccrs.SouthPolarStereo()})
for i, var in enumerate(tqdm(var_info.keys())):
    for j, season in enumerate(tqdm(seasons.keys(), leave=False)):
        ax = axes[i][j]
        ax.set_extent([-180, 180, -90, -40], ccrs.PlateCarree())
        ds = xr.open_dataset('../data/in/bsose/bsose_i133_2013to2018_monthly_' + var_info.get(var)['file_prefix'] + '.nc')
        if var == 'SIarea':
          ds = ds[var][seasons.get(season), :, :].mean(dim='time')
        else:
          ds = ds[var][seasons.get(season), 7, :, :].mean(dim='time')
          ds = ds.where(ds > var_info.get(var)['vmin'])
        ds.plot.pcolormesh(x="XC", y="YC", ax=ax, transform=ccrs.PlateCarree(), cmap=var_info.get(var)['cmap'])
        ax.add_feature(cartopy.feature.LAND, zorder=2)
        # ds.to_netcdf(f'../data/maxent/in/bsose_processed/netcdf/{var_info.get(var)["simple_name"]}_{season}.nc')

## Plotting processed B-SOSE environmental layers

In [ ]:
# TODO: make shared colorbar for each variable and season labels

# Open the newly processed datasets and plot them
fig, axes = plt.subplots(len(var_info), len(seasons), figsize=(16, 16), subplot_kw={'projection': ccrs.SouthPolarStereo()})
for i, var in enumerate(tqdm(var_info.keys())):
    for j, season in enumerate(tqdm(seasons.keys(), leave=False)):
        ax = axes[i][j]
        ax.set_extent([-180, 180, -90, -40], ccrs.PlateCarree())
        ds = xr.open_dataset(f'../data/maxent/in/bsose_processed/netcdf/{var_info.get(var)["simple_name"]}_{season}.nc')
        ds[var].plot.pcolormesh(x="XC", y="YC", ax=ax, transform=ccrs.PlateCarree(), cmap=var_info.get(var)['cmap'])
        ax.add_feature(cartopy.feature.LAND, zorder=2)